In [1]:
#pandas libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import random

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import Normalizer

import statistics

from scipy.stats import uniform

%run "../notebooks/cargar_df.ipynb"

### Cargo los data frames con los features

In [2]:
trainX = pd.read_csv('../files_csv/train_173_features_bin_encoding')
trainX.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD,Stage
0,0,1,1,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0.52,0.58817,5.609362e+06,0
1,1,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0.53,0.59948,5.130851e+04,1
2,2,0,0,1,1,0,1,0,1,0,...,1,0,0,0,1,0,0.48,0.48000,8.386560e+04,1
3,3,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53,0.53000,7.421882e+06,0
4,4,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53,0.53000,1.335719e+07,0


In [3]:
stages = trainX['Stage']
stages.head()

0    0
1    1
2    1
3    0
4    0
Name: Stage, dtype: int64

In [4]:
ids = trainX.Opportunity_ID
ids.head()

0    0
1    1
2    2
3    3
4    4
Name: Opportunity_ID, dtype: int64

In [5]:
trainX = trainX.drop(['Opportunity_ID', 'Stage'], axis=1)

In [6]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 12.8 MB


In [7]:
testX = pd.read_csv('../files_csv/test_173_features_bin_encoding')
testX.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_1,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD
0,10689,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.306667,0.346870,137147.816349
1,10690,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.332000,0.375524,169716.349384
2,10691,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.425000,0.425000,21037.500000
3,10692,1,1,1,0,1,1,0,0,1,...,0,1,0,0,0,1,0,0.425000,0.425000,361517.750000
4,10693,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.390000,0.390000,5752.500000


In [8]:
ids_test = testX.Opportunity_ID
ids_test.head()

0    10689
1    10690
2    10691
3    10692
4    10693
Name: Opportunity_ID, dtype: int64

In [9]:
testX = testX.drop('Opportunity_ID', axis=1)

In [10]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 2.1 MB


# Random Forest

In [11]:
# Del archivo 'prediccion_log_regression'
# Las columnas mas influyentes para predecir con Logistic Regression

test_size = 0.3
random_state = 0
n_jobs=-1
n_estimators = 100
oob_score = True

In [12]:
datos_train , datos_test, cat_train, cat_test = train_test_split(trainX,
                                                                stages,
                                                                test_size = test_size,
                                                                random_state = random_state)

In [13]:
clf = RandomForestClassifier(
    n_jobs = n_jobs, 
    random_state = random_state, 
    oob_score = oob_score, 
    n_estimators = n_estimators,
    max_depth = 100,
    criterion = 'gini',
    min_samples_split = 10,
    min_samples_leaf = 3,
    max_features = None
)

In [14]:
clf.fit(datos_train,cat_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features=None,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=0, verbose=0,
                       warm_start=False)

In [15]:
preds = clf.predict(datos_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

rmse = log_loss(cat_test, preds, normalize=True)
print("Log Loss: %f" % (rmse))

NameError: name 'cat_pred' is not defined

In [ ]:
final_test_data = testX[trainX.columns]
preds = clf.predict(final_test_data)
prediccion = pd.DataFrame({'Opportunity_ID':ids_test, 'Target':pd.DataFrame(preds)[0]})
prediccion.head()

In [ ]:
prediccion.to_csv('prediccion_random_forest', index=False)

In [ ]:
# Calculo el error
def calcular_error_rand_forest(df):
    kf = KFold(n_splits=5, shuffle=True)

    errores = []
    for train_index, test_index in kf.split(df):
        X_train, X_test = df.iloc[train_index], df.iloc[test_index]
        y_train, y_test = stages.iloc[train_index], stages.iloc[test_index]
        rfc = RandomForestClassifier(
                n_jobs = n_jobs, 
                random_state = random_state, 
                oob_score = oob_score, 
                n_estimators = n_estimators,
                max_depth = 10,
                criterion = 'gini',
                min_samples_split = 5,
                min_samples_leaf = 3,
                max_features = None
        )
        rfc.fit(X_train, y_train)
        preds = rfc.predict(X_test)

        loss = log_loss(y_test, preds)
        errores.append(loss)
    
    print("Log loss: %f" % (statistics.mean(errores)))

In [ ]:
calcular_error_rand_forest(trainX)

# Busqueda de hiperparametros optimos

In [ ]:

rf = RandomForestClassifier(oob_score=oob_score, random_state=random_state, n_jobs=n_jobs)

param_grid = { 
    "criterion" : ["gini", "entropy"], 
    "min_samples_leaf" : [1, 5, 10], 
    "min_samples_split" : [2, 4, 10, 12, 16], 
    "n_estimators": [50, 100, 400, 700, 1000],
    'max_depth': [1, 10, 50, 100]
}

gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=n_jobs)

gs = gs.fit(trainX.iloc[:, 1:], trainX.iloc[:, 0])

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
gs.cv_results_

In [ ]:
# nuevos hiperparametros que supuestamente optimizan el modelo
criterion = 'gini'
min_samples_leaf = 1
min_samples_split = 4
n_estimators = 1000

In [ ]:
datos_train , datos_test, cat_train, cat_test = train_test_split(trainX,
                                                                stages,
                                                                test_size = test_size,
                                                                random_state = random_state)

In [ ]:
clf = RandomForestClassifier(
    criterion = criterion, 
    min_samples_leaf = min_samples_leaf, 
    min_samples_split = min_samples_split, 
    n_estimators = n_estimators,
    oob_score = oob_score,
    random_state = random_state,
    n_jobs = n_jobs)

In [ ]:
clf.fit(datos_train,cat_train)

In [ ]:
preds = clf.predict(datos_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,preds)*100))

rmse = log_loss(cat_test, preds, normalize=True)
print("Log Loss: %f" % (rmse))

In [ ]:
final_test_data = testX[trainX.columns]
preds = clf.predict(final_test_data)
prediccion = pd.DataFrame({'Opportunity_ID':ids_test, 'Target':pd.DataFrame(preds)[0]})
prediccion.to_csv('prediccion_random_forest', index=False)